In [81]:
from dotenv import load_dotenv
import requests
import os
from bs4 import BeautifulSoup
import psycopg2

In [82]:
load_dotenv()

api_key = os.getenv("ALPHA_VANTAGE_API_KEY")

railway_db = os.getenv("MY_RAILWAY_KEY")

In [83]:
def get_bitcoin_data():
    url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey=" + api_key
    response_00 = requests.get(url)
    values_00 = response_00.json()

    print(values_00)

    time_series_daily = values_00.get("Time Series (Daily)", {})

    bitcoin_data = []
    for date, data in time_series_daily.items():
        closing_price = data.get("4. close", "N/A")
        bitcoin_data.append({"Date": date, "Closing_Price": closing_price})
    
    return bitcoin_data
       
get_bitcoin_data()


{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'IBM', '3. Last Refreshed': '2025-02-03', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2025-02-03': {'1. open': '252.4000', '2. high': '262.0600', '3. low': '251.8400', '4. close': '260.7300', '5. volume': '8408523'}, '2025-01-31': {'1. open': '256.0500', '2. high': '257.2350', '3. low': '251.8900', '4. close': '255.7000', '5. volume': '7203519'}, '2025-01-30': {'1. open': '250.0000', '2. high': '261.8000', '3. low': '247.0100', '4. close': '258.2700', '5. volume': '15381890'}, '2025-01-29': {'1. open': '225.6200', '2. high': '229.4700', '3. low': '223.7300', '4. close': '228.6300', '5. volume': '7079804'}, '2025-01-28': {'1. open': '224.3200', '2. high': '225.7700', '3. low': '221.7700', '4. close': '225.6600', '5. volume': '4485429'}, '2025-01-27': {'1. open': '222.1900', '2. high': '224.3000', '3. low': '219.8400', '4. close': '224.1300', '5. vol

[{'Date': '2025-02-03', 'Closing_Price': '260.7300'},
 {'Date': '2025-01-31', 'Closing_Price': '255.7000'},
 {'Date': '2025-01-30', 'Closing_Price': '258.2700'},
 {'Date': '2025-01-29', 'Closing_Price': '228.6300'},
 {'Date': '2025-01-28', 'Closing_Price': '225.6600'},
 {'Date': '2025-01-27', 'Closing_Price': '224.1300'},
 {'Date': '2025-01-24', 'Closing_Price': '224.8000'},
 {'Date': '2025-01-23', 'Closing_Price': '226.0400'},
 {'Date': '2025-01-22', 'Closing_Price': '223.2600'},
 {'Date': '2025-01-21', 'Closing_Price': '224.2600'},
 {'Date': '2025-01-17', 'Closing_Price': '224.7900'},
 {'Date': '2025-01-16', 'Closing_Price': '222.6600'},
 {'Date': '2025-01-15', 'Closing_Price': '220.0300'},
 {'Date': '2025-01-14', 'Closing_Price': '217.7500'},
 {'Date': '2025-01-13', 'Closing_Price': '217.4000'},
 {'Date': '2025-01-10', 'Closing_Price': '219.7500'},
 {'Date': '2025-01-08', 'Closing_Price': '223.1800'},
 {'Date': '2025-01-07', 'Closing_Price': '223.9600'},
 {'Date': '2025-01-06', 'Clo

In [84]:
def get_bitcoin_news():
    
    url = "https://www.ft.com/search?q=bitcoin"
    
    response_01 = requests.get(url)
    
    soup = BeautifulSoup(response_01.text, "html.parser")

    articles = soup.find_all("a", class_= "js-teaser-heading-link")

    article_data = []

    for article in articles:
        title = article.text.strip()
        link = "https://www.ft.com" + article["href"]
        article_data.append({"Title": title, "Link": link})
    
    return article_data

get_bitcoin_news()
    

[{'Title': 'Bitcoin miners stockpile coins to ride out profit squeeze',
  'Link': 'https://www.ft.comhttps://www.ft.com/content/0cfbe43c-7c2a-40bf-8bb7-92f58a9d2389'},
 {'Title': 'Bitcoin hit $100,000. What’s next?',
  'Link': 'https://www.ft.comhttps://www.ft.com/content/11148764-9692-41a1-bda6-cc47e2c983fa'},
 {'Title': 'Bitcoin hits $100,000 as Trump era hopes grow',
  'Link': 'https://www.ft.comhttps://www.ft.com/content/f8db431e-57f9-4b43-8873-263101d070a5'},
 {'Title': 'Crypto is celebrating but Trump’s boosterism could end badly',
  'Link': 'https://www.ft.comhttps://www.ft.com/content/8d302c79-8912-4f8e-bb1a-eea77ea0d99e'},
 {'Title': 'Bitcoin hits record above $75,000 as crypto industry celebrates Trump win',
  'Link': 'https://www.ft.comhttps://www.ft.com/content/bdc3311d-3a29-4a3a-82e7-d646bc065dce'},
 {'Title': 'Czech central bank head wants it to buy billions of euros in bitcoin',
  'Link': 'https://www.ft.comhttps://www.ft.com/content/a3c06f8f-34ad-4065-bcf4-97670230824f'

In [85]:
def save_to_railway(bitcoin_data, article_data):

    try:

        conn = psycopg2.connect(railway_db)
        cursor = conn.cursor()

        cursor.execute('''CREATE TABLE IF NOT EXISTS bitcoin_data (
            id SERIAL PRIMARY KEY,
            date DATE UNIQUE,
            closing_price NUMERIC
        );
        ''')

        cursor.execute('''CREATE TABLE IF NOT EXISTS article_data (
            id SERIAL PRIMARY KEY,
            title TEXT UNIQUE,
            link TEXT
        );
        ''')

    
        for item in bitcoin_data:
            date = item["Date"]
            closing_price = item["Closing_Price"]
            cursor.execute("""
                INSERT INTO bitcoin_data (date, closing_price)
                VALUES (%s, %s)
                ON CONFLICT (date) DO UPDATE 
                SET closing_price = EXCLUDED.closing_price;
                """, (date, closing_price))

        for article in article_data:
            title = article["Title"]
            link = article["Link"]
            cursor.execute("""
                INSERT INTO article_data (title, link)
                VALUES (%s, %s)
                ON CONFLICT (title) DO UPDATE 
                SET link = EXCLUDED.link;
                """, (title, link))


        conn.commit()
        cursor.close()
        conn.close()
        print("data saved :) ")

    except Exception as e:
        print(f"❌ Error: {e}")



bitcoin_data = get_bitcoin_data()  
news_articles = get_bitcoin_news()  

print("Bitcoin Data:", bitcoin_data)
print("News Articles:", news_articles)

if bitcoin_data and news_articles:
    print("Data available, saving to database...")
    save_to_railway(bitcoin_data, news_articles)
else:
    print("❌ No data available.")


{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'IBM', '3. Last Refreshed': '2025-02-03', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2025-02-03': {'1. open': '252.4000', '2. high': '262.0600', '3. low': '251.8400', '4. close': '260.7300', '5. volume': '8408523'}, '2025-01-31': {'1. open': '256.0500', '2. high': '257.2350', '3. low': '251.8900', '4. close': '255.7000', '5. volume': '7203519'}, '2025-01-30': {'1. open': '250.0000', '2. high': '261.8000', '3. low': '247.0100', '4. close': '258.2700', '5. volume': '15381890'}, '2025-01-29': {'1. open': '225.6200', '2. high': '229.4700', '3. low': '223.7300', '4. close': '228.6300', '5. volume': '7079804'}, '2025-01-28': {'1. open': '224.3200', '2. high': '225.7700', '3. low': '221.7700', '4. close': '225.6600', '5. volume': '4485429'}, '2025-01-27': {'1. open': '222.1900', '2. high': '224.3000', '3. low': '219.8400', '4. close': '224.1300', '5. vol